In [ ]:
%%capture
!pip install yfinance
import numpy as np
import pandas as pd
import yfinance as yf

In [ ]:
saudi_tickers = [1010,1020,1030,1050,1060,1080,1111,1120,1140,1150,1180,1182,1183,1201,1202,1210,1211,1212,1213,1214,1301,
                 1302,1303,1304,1320,1321,1322,1810,1820,1830,1831,1832,1833,1834,1835,2001,2010,2020,2030,2040,2050,2060,
                 2070,2080,2081,2082,2083,2084,2090,2100,2110,2120,2130,2140,2150,2160,2170,2180,2190,2200,2210,2220,2222,
                 2223,2230,2240,2250,2270,2280,2281,2282,2283,2284,2285,2286,2290,2300,2310,2320,2330,2340,2350,2360,2370,
                 2380,2381,2382,3002,3003,3004,3005,3007,3008,3010,3020,3030,3040,3050,3060,3080,3090,3091,3092,4001,4002,
                 4003,4004,4005,4006,4007,4008,4009,4011,4012,4013,4014,4015,4016,4017,4018,4020,4030,4031,4040,4050,4051,
                 4061,4070,4071,4072,4080,4081,4082,4083,4090,4100,4110,4130,4140,4141,4142,4143,4150,4160,4161,4162,4163,
                 4164,4165,4170,4180,4190,4191,4192,4193,4200,4210,4220,4230,4240,4250,4260,4261,4262,4263,4270,4280,4290,
                 4291,4292,4300,4310,4320,4321,4322,4323,4324,5110,6001,6002,6004,6010,6012,6013,6014,6016,6017,6020,6040,
                 6050,6060,6070,6090,7010,7020,7030,7040,7200,7201,7202,7203,7204,8010,8012,8020,8030,8040,8050,8060,8070,
                 8100,8120,8150,8160,8170,8180,8190,8200,8210,8230,8240,8250,8260,8270,8280,8300,8310,8311,8313,9510,9513,
                 9514,9515,9516,9517,9518,9521,9522,9523,9524,9525,9527,9528,9529,9530,9531,9532,9533,9534,9535,9536,9537,
                 9538,9539,9540,9541,9542,9543,9544,9545,9546,9547,9548,9549,9550,9551,9552,9553,9555,9556,9557,9558,9559,
                 9560,9561,9562,9563,9564,9565,9566,9567,9568,9569,9570,9571,9572,9574,9575,9577,9578,9579,9580,9581,9583,
                 9584,9585,9586,9587,9588,9589,9590,9591,9592,9593,9594,9595,9596,9597,9598,9599,9600,9601,9602,9603,9604,
                 9605,9606,9607,9608,9609,9610,9611,9612,9613,9614,9615,9616,9617,9618,9619,9620,9621,9622,9623,9625]
for numerical_part in saudi_tickers:
  ticker = str(numerical_part)
  ticker += '.'
  ticker += 'S'
  ticker += 'R'
  data = yf.download(ticker, start="2010-01-01", end="2025-01-24")
  data["prv_low_1"] = data["Low"].shift(1)
  data["prv_low_2"] = data["Low"].shift(2)
  data["prv_low_3"] = data["Low"].shift(3)
  data["prv_low_4"] = data["Low"].shift(4)
  data["prv_low_5"] = data["Low"].shift(5)
  data["Low_Last_5d"] = data[["prv_low_1", "prv_low_2","prv_low_3","prv_low_4","prv_low_5"]].values.min(1)
  data = data.drop(["prv_low_1", "prv_low_2", "prv_low_3", "prv_low_4", "prv_low_5"], axis=1)
  data['prv_high_1'] = data["High"].shift(1)
  data['prv_high_2'] = data["High"].shift(2)
  data['prv_high_3'] = data["High"].shift(3)
  data['prv_high_4'] = data["High"].shift(4)
  data['prv_high_5'] = data["High"].shift(5)
  data["High_Last_5d"] = data[["prv_high_1", "prv_high_2","prv_high_3","prv_high_4","prv_high_5"]].values.max(1)
  data = data.drop(["prv_high_1", "prv_high_2", "prv_high_3", "prv_high_4", "prv_high_5"], axis=1)
  data["seven_percent_of_close_price"] = .93*data["Close"]
  data["Close"] = 1*data["Close"]
  data["seven_percent_of_close_price_within_last_five_days"] = np.where((data['Low_Last_5d'] <= data['seven_percent_of_close_price']) & (data['seven_percent_of_close_price'] <= data['High_Last_5d']), True, False)
  data["Last_Close"] = data["Close"].shift(1)
  data["close_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Last_Close"].values.reshape(data.shape[0])
  data["up_from_yesterday"] = np.where((data["close_diff"] > 0), True, False)
  data["buy"] = data["seven_percent_of_close_price_within_last_five_days"] & data["up_from_yesterday"]
  data["buycumsum"] = data["buy"].cumsum().astype('int64')
  data = data.drop(["Last_Close","close_diff","seven_percent_of_close_price"], axis=1)
  data["Close_up1%"] = 1.01*data["Close"]
  data["Close_down1%"] = .99*data["Close"]
  data.loc[(data.buy == False), 'Close_up1%'] = np.nan
  data.loc[(data.buy == False), 'Close_down1%'] = np.nan
  data["Close_up1%"] = data["Close_up1%"].ffill()
  data["Close_down1%"] = data["Close_down1%"].ffill()
  data = data.drop(["Low_Last_5d", "High_Last_5d", "seven_percent_of_close_price_within_last_five_days", "up_from_yesterday", "High", "Low"], axis=1)
  data["close_up_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_up1%"].values.reshape(data.shape[0])
  data["closeaboveup1%"] = np.where((data["close_up_diff"] > 0), True, False)
  data["close_down_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_down1%"].values.reshape(data.shape[0])
  data["closebelowdown1%"] = np.where((data["close_down_diff"] < 0), True, False)
  data = data.drop(["close_up_diff", "close_down_diff"], axis=1)
  data["aboveup1%_or_belowdown1%"] = np.where((data["closeaboveup1%"]) | (data["closebelowdown1%"]), True, False)
  #data.tail(3)
  data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
  #data.head(3)
  data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
  #data.head(3)

  # exit and sell stock everytime a one percent loss is realized or exit and sell stock everytime a one percent gain is realized
  data.to_csv('out.csv', index=False)
  data = pd.read_csv('out.csv')
  data = data.drop_duplicates(subset="buycumsum",inplace=False)
  print("\n\n"+ticker)
  print("\n"+str(data["closeaboveup1%"].sum()))
  print("\n"+str(data["closebelowdown1%"].sum()))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



1010.SR

13

68


1020.SR

12

97



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



1030.SR

10

58




1050.SR

12

115


...



9621.SR

0.0

0.0


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



9622.SR

0

1



[*********************100%***********************]  1 of 1 completed



9623.SR

0

2


9625.SR

0.0

0.0
